In [1]:
from nuscenes import NuScenes
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.detection.data_classes import DetectionBox 
from pyquaternion import Quaternion
import json

In [2]:
nusc = NuScenes(version='v1.0-mini', dataroot='/home/mqdao/Downloads/nuScene/v1.0-mini', verbose=False)

In [3]:
my_scene_token = nusc.field2token('scene', 'name', 'scene-0757')[0]
my_scene = nusc.get('scene', my_scene_token)
my_scene

{'token': '2fc3753772e241f2ab2cd16a784cc680',
 'log_token': '3a43824b84534c98bda1d07548db5817',
 'nbr_samples': 41,
 'first_sample_token': 'cd9964f8c3d34383b16e9c2997de1ed0',
 'last_sample_token': '8fe9664cec514a58b1184c4fcefda6b5',
 'name': 'scene-0757',
 'description': 'Arrive at busy intersection, bus, wait at intersection, bicycle, peds'}

# Process detection file

In [4]:
detection_file = '/home/mqdao/Downloads/nuScene/detection-megvii/megvii_train.json'

In [5]:
with open(detection_file) as f:
    data = json.load(f)
all_results = EvalBoxes.deserialize(data['results'], DetectionBox)
meta = data['meta']
print('meta: ', meta)
print("Loaded results from {}. Found detections for {} samples."
.format(detection_file, len(all_results.sample_tokens)))

meta:  {'use_camera': False, 'use_lidar': True, 'use_radar': False, 'use_map': False, 'use_external': False}
Loaded results from /home/mqdao/Downloads/nuScene/detection-megvii/megvii_train.json. Found detections for 28130 samples.


# Extract detection for 1 scene from Detection file

In [7]:
NUSCENES_TRACKING_NAMES = [
  'bicycle',
  'bus',
  'car',
  'motorcycle',
  'pedestrian',
  'trailer',
  'truck'
]

In [8]:
# store all measurements
all_measurements = {}
all_classes = {}
sample_token = my_scene['first_sample_token']
current_time_step = 0
score_theshold = 0.2
while True:
    sample_record = nusc.get('sample', sample_token)
    gt_num_objects = len(sample_record['anns'])
    # filter detections with low detection score
    filtered_detections = []
    i_threshold = 0
    filtered_detections = [detection for detection in all_results.boxes[sample_token]
                           if detection.detection_score > score_theshold]
    all_measurements[current_time_step] = []
    all_classes[current_time_step] = []
    for detection in filtered_detections:
        # get obj_type
        if detection.detection_name not in NUSCENES_TRACKING_NAMES:
            continue
        obj_type = detection.detection_name
        # get object pose
        x, y, z = detection.translation
        quaternion = Quaternion(detection.rotation)
        yaw = quaternion.angle if quaternion.axis[2] > 0 else -quaternion.angle 
        # get size
        size = list(detection.size)
        # get score
        score = detection.detection_score
        # store mearurement & class
        all_measurements[current_time_step].append([x, y, yaw] + size + [z] + [score])
        all_classes[current_time_step].append(obj_type)
    
    # move on
    sample_token = sample_record['next']
    if sample_token == '':
        break
    current_time_step += 1

In [9]:
print('len(all_measurements)', len(all_measurements))
print('len(all_classes)', len(all_classes))

len(all_measurements) 41
len(all_classes) 41


In [10]:
print('len(all_measurements[0])', len(all_measurements[0]))

len(all_measurements[0]) 12


## Save scene measurements to file

In [11]:
outdata = {'all_measurements': all_measurements,
         'all_classes': all_classes}
with open('./../scene-detection/fixed-megvii-meausrement-full-scene-0757.json', 'w') as outfile:
    json.dump(outdata, outfile)